Let's begin by import dependencies

In [1]:
import numpy as np
import boostrtrees as brt

Now let's instantiate the **RTree3D** object

In [2]:
rt3 = brt.RTree3D()

We'll create a set of points we'll be using to test the RTree as Python tuples. For simplicity I'm going to add three corners of a **10x10x10** cube

In [3]:
cube_corners = [(5.0, 5.0, 5.0), (5.0, 5.0, -5.0), (5.0, -5.0, 5.0)]

Let us insert these points into the R-Tree

In [4]:
for index, corner in enumerate(cube_corners):
    rt3.insert_point(x=corner[0], y=corner[1], z=corner[2], value=index)

We can verify that the points have been inserted by checking the `size()`

In [5]:
rt3.size()

3

You can also insert points into the RTree in the form of a 2-dimensional NumPy array of shape **(N, 4)**, where N is the number of points being added.

For example,

In [6]:
other_corners = [(5.0, -5.0, -5.0),\
                 (-5.0, 5.0, 5.0), \
                 (-5.0, 5.0, -5.0),\
                 (-5.0, -5.0, 5.0),\
                 (-5.0, -5.0, -5.0)]

other_corners_np = np.array([ _tc + tuple([index+len(cube_corners)]) for index, _tc in enumerate(other_corners) ])
print(other_corners_np)

[[ 5. -5. -5.  3.]
 [-5.  5.  5.  4.]
 [-5.  5. -5.  5.]
 [-5. -5.  5.  6.]
 [-5. -5. -5.  7.]]


In [7]:
rt3.insert_points(other_corners_np)

In [8]:
rt3.size()

8

Now, we have added all 8 corners of the 10x10x10 cube into our 3-dimensional R-Tree!

Let's check to see if the RTree works as expected :)

### Nearest Neighbor queries

Fast Nearest neighbor queries are about the most useful feature of RTrees. Let's see if we can find the **2 nearest neighbors** of the point **(5.0, 4.0, 0.0)**.

If this works corectly, we expect it to return the indices of **(5.0, 5.0, 5.0)** and **(5.0, 5.0, -5.0)**, which are 0 and 1

In [9]:
rt3.knn(x=5.0, y=5.0, z=5.0, k=2)

[0, 1]

... and Bingo!

Now, because we all love NumPy so much, we added a function that accepts coordinates as a NumPy array.

### Bounds

This is how you get the minimum enclosing bounding box _in 3D_ of the points

In [10]:
bounds = rt3.bounds()
print(bounds)

{'min_x': -5.0, 'max_x': 5.0, 'min_y': -5.0, 'max_y': 5.0, 'min_z': -5.0, 'max_z': 5.0}


So what happens to the bounding box if we add a point outside the **10x10x10** cube?

In [11]:
rt3.insert_point(x=6.0, y=6.0, z=6.0, value=rt3.size())

In [12]:
bounds = rt3.bounds()
print(bounds)

{'min_x': -5.0, 'max_x': 6.0, 'min_y': -5.0, 'max_y': 6.0, 'min_z': -5.0, 'max_z': 6.0}


As you would expect, the 3D bounding box expands to include that point!

### Intersection
Get points inersecting a cuboid _(enclosed by the 3D bounding box)_

**Note:** The coordinates are non-interleaved i.e of the form (min_x, min_y, min_z, max_x, max_y, max_z)

Let's try to get all points within the box between these two corner points: **(4.0, 4.0, 4.0)** and **(7.0, 7.0, 7.0)**

In [14]:
min_point = (4.0, 4.0, 4.0)
max_point = (7.0, 7.0, 7.0)

rt3.intersection(np.array([min_point, max_point]))

[8, 0]